# Week 14: CNN Lab - Rock, Paper, Scissors

**Objective:** Build, train, and test a Convolutional Neural Network (CNN) to classify images of hands playing Rock, Paper, or Scissors.

### Step 1: Setup and Data Download

This first cell downloads the dataset from Kaggle.

In [24]:
# Download dataset using the official Kaggle API (preferred)
# This cell will try to install the `kaggle` package if missing,
# then download and unzip the 'drgfreeman/rockpaperscissors' dataset.
import sys
import subprocess
import os

# Install kaggle if it's not available
try:
    import kaggle
except Exception:
    print('`kaggle` package not found; installing...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'kaggle'])
    import kaggle

# Two common ways to provide credentials:
# 1) Set KAGGLE_USERNAME and KAGGLE_KEY environment variables, OR
# 2) Place your `kaggle.json` in the ~/.kaggle/ directory with mode 600.
if (('KAGGLE_USERNAME' not in os.environ) or ('KAGGLE_KEY' not in os.environ)) and not os.path.exists(os.path.expanduser('~/.kaggle/kaggle.json')):
    print('Kaggle credentials not found. Set `KAGGLE_USERNAME` and `KAGGLE_KEY` env vars, or put kaggle.json in ~/.kaggle/.')
    print('See: https://github.com/Kaggle/kaggle-api#api-credentials for details')
else:
    from kaggle.api.kaggle_api_extended import KaggleApi
    api = KaggleApi()
    api.authenticate()
    # download and unzip into ./dataset_rps (safe isolated folder)
    out_dir = 'dataset_rps'
    os.makedirs(out_dir, exist_ok=True)
    print('Downloading dataset to', out_dir)
    api.dataset_download_files('drgfreeman/rockpaperscissors', path=out_dir, unzip=True)
    print('Download complete. Extracted into', out_dir)
    DATA_DIR = os.path.abspath(out_dir)
    print('Path to dataset files:', DATA_DIR)

# If you prefer not to use the Kaggle API, you can manually upload the dataset
# to the environment and set DATA_DIR to the upload path.


Dataset URL: https://www.kaggle.com/datasets/drgfreeman/rockpaperscissors


Download complete. Extracted into dataset_rps
Path to dataset files: /home/mona/ML_F_PES2UG23CS906_MonishaSharma/Lab14/dataset_rps


In [25]:
import shutil
import os

# Use a workspace-relative destination folder so we don't need /content (may be unwritable)
src_root = "/kaggle/input/rockpaperscissors"
dst_root = os.path.join(os.getcwd(), 'dataset')

try:
    os.makedirs(dst_root, exist_ok=True)
except PermissionError:
    # Fallback: use a local subfolder inside current working directory with a different name
    dst_root = os.path.join(os.getcwd(), 'dataset_fallback')
    os.makedirs(dst_root, exist_ok=True)
    print('Permission denied for initial dst; using', dst_root)

folders_to_copy = ["rock", "paper", "scissors"]

for folder in folders_to_copy:
    src_path = os.path.join(src_root, folder)
    dst_path = os.path.join(dst_root, folder)

    if os.path.exists(src_path):
        shutil.copytree(src_path, dst_path, dirs_exist_ok=True)
        print("Copied:", folder)
    else:
        print("Folder not found in /kaggle/input (this is normal unless running in Kaggle):", folder)

print('Final dataset destination:', dst_root)


Folder not found in /kaggle/input (this is normal unless running in Kaggle): rock
Folder not found in /kaggle/input (this is normal unless running in Kaggle): paper
Folder not found in /kaggle/input (this is normal unless running in Kaggle): scissors
Final dataset destination: /home/mona/ML_F_PES2UG23CS906_MonishaSharma/Lab14/dataset


In [26]:
# Ensure torch and torchvision are available. If missing, install CPU-only wheels.
import sys, subprocess
try:
    import torch
    import torchvision
    print('Found torch', torch.__version__, 'and torchvision', torchvision.__version__)
except Exception as e:
    print('torch/torchvision not found or import failed:', e)
    print('Installing torch, torchvision (CPU builds). This may take a few minutes...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'torch', 'torchvision', 'torchaudio', '--index-url', 'https://download.pytorch.org/whl/cpu'])
    import torch
    import torchvision
    print('Installed torch', torch.__version__, 'and torchvision', torchvision.__version__)


Found torch 2.9.0+cu128 and torchvision 0.24.1+cpu


### Step 2: Imports and Device Setup

Import the necessary libraries and check if a GPU is available.

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Using device:", device)

Using device: cpu


### Step 3: Data Loading and Preprocessing

Here we will define our image transformations, load the dataset, split it, and create DataLoaders.

In [ ]:
# Prefer the Kaggle-downloaded folder if present, otherwise prefer a local 'dataset' folder
possible = ["dataset_rps", os.path.join(os.getcwd(), 'dataset'), os.path.join(os.getcwd(), 'dataset_fallback'), "/content/dataset"]
DATA_DIR = None
for p in possible:
    if os.path.exists(p):
        DATA_DIR = p
        break
if DATA_DIR is None:
    # If nothing exists yet, default to a workspace-relative 'dataset' and create it
    DATA_DIR = os.path.join(os.getcwd(), 'dataset')
    os.makedirs(DATA_DIR, exist_ok=True)
    print('Created workspace dataset folder:', DATA_DIR)
else:
    print('Using DATA_DIR =', DATA_DIR)


transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize to 128x128
    transforms.ToTensor(),           # Convert to tensor
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])

# Load dataset using ImageFolder
full_dataset = datasets.ImageFolder(DATA_DIR, transform=transform)

class_names = full_dataset.classes
print("Classes:", class_names)
# Number of classes discovered by ImageFolder (used to set model output size)
num_classes = len(class_names)
print('num_classes =', num_classes)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size

train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Total images: {len(full_dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Test images: {len(test_dataset)}")

Using DATA_DIR = dataset_rps
Classes: ['paper', 'rock', 'scissors']
num_classes = 3
Total images: 2188
Training images: 1750
Test images: 438


In [34]:
# Diagnostic: List all subdirectories in DATA_DIR
import os
all_items = os.listdir(DATA_DIR)
all_dirs = [d for d in all_items if os.path.isdir(os.path.join(DATA_DIR, d))]
print('All subdirectories in DATA_DIR:', all_dirs)
print('Expected classes: rock, paper, scissors')
print('Actual classes found by ImageFolder:', full_dataset.classes)
if len(all_dirs) != 3 or set(full_dataset.classes) != {'rock', 'paper', 'scissors'}:
    print('WARNING: Unexpected classes detected!')
    print('Extra folders:', [d for d in all_dirs if d not in ['rock', 'paper', 'scissors']])
    print('ACTION: Remove or move extra folders before re-running the model cells.')
else:
    print('✓ All classes correct!')

All subdirectories in DATA_DIR: ['paper', 'scissors', 'rock', 'rps-cv-images']
Expected classes: rock, paper, scissors
Actual classes found by ImageFolder: ['paper', 'rock', 'rps-cv-images', 'scissors']
Extra folders: ['rps-cv-images']
ACTION: Remove or move extra folders before re-running the model cells.


In [ ]:
# Clean up: Remove or move unexpected class folders
import shutil
expected_classes = {'rock', 'paper', 'scissors'}
unexpected = [d for d in all_dirs if d not in expected_classes]
if unexpected:
    print(f'Found unexpected folders: {unexpected}')
    for folder in unexpected:
        folder_path = os.path.join(DATA_DIR, folder)
        backup_path = folder_path + '_backup'
        shutil.move(folder_path, backup_path)
        print(f'Moved {folder} to {backup_path}')
    print('Re-run the data loading cell (Step 3) to reload the dataset with correct classes.')
else:
    print('✓ No unexpected folders found.')

Found unexpected folders: ['rps-cv-images']
Moved rps-cv-images to dataset_rps/rps-cv-images_backup
Re-run the data loading cell (Step 3) to reload the dataset with correct classes.


### Step 4: Define the CNN Model

Fill in the `conv_block` and `fc_block` with the correct layers.

In [ ]:
class RPS_CNN(nn.Module):
    def __init__(self, num_classes=3):
        super(RPS_CNN, self).__init__()

        self.conv_block = nn.Sequential(
            # Block 1: 3 -> 16 channels
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 2: 16 -> 32 channels
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            # Block 3: 32 -> 64 channels
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Sequential(
            nn.Flatten(),                    # Flatten the input
            nn.Linear(64 * 16 * 16, 256),   # 64 channels * 16 * 16 spatial dims
            nn.ReLU(),
            nn.Dropout(p=0.3),
            nn.Linear(256, num_classes)               # output classes = num_classes
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc(x)
        return x

model = RPS_CNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print(model)

RPS_CNN(
  (conv_block): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=16384, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=3, bias=True)
  )
)


### Step 5: Train the Model

Fill in the core training steps inside the loop.

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{EPOCHS}, Loss = {total_loss/len(train_loader):.4f}")

print("Training complete!")

Epoch 1/10, Loss = 0.5635
Epoch 2/10, Loss = 0.1861
Epoch 2/10, Loss = 0.1861
Epoch 3/10, Loss = 0.0692
Epoch 3/10, Loss = 0.0692
Epoch 4/10, Loss = 0.0423
Epoch 4/10, Loss = 0.0423
Epoch 5/10, Loss = 0.0183
Epoch 5/10, Loss = 0.0183
Epoch 6/10, Loss = 0.0153
Epoch 6/10, Loss = 0.0153
Epoch 7/10, Loss = 0.0172
Epoch 7/10, Loss = 0.0172
Epoch 8/10, Loss = 0.0185
Epoch 8/10, Loss = 0.0185
Epoch 9/10, Loss = 0.0181
Epoch 9/10, Loss = 0.0181
Epoch 10/10, Loss = 0.0031
Training complete!
Epoch 10/10, Loss = 0.0031
Training complete!


### Step 6: Evaluate the Model

Test the model's accuracy on the unseen test set.

In [ ]:
model.eval() # Set the model to evaluation mode
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 99.09%


### Step 7: Test on a Single Image

Let's see how the model performs on one image.

In [ ]:
def predict_image(model, img_path):
    model.eval()

    img = Image.open(img_path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        _, pred = torch.max(output, 1)

    return class_names[pred.item()]

# Test the function
import os
test_img_path = os.path.join(DATA_DIR, 'paper', '0Uomd0HvOB33m47I.png')
prediction = predict_image(model, test_img_path)
print(f"Model prediction for {test_img_path}: {prediction}")

Model prediction for dataset_rps/paper/0Uomd0HvOB33m47I.png: paper


### Step 8: Play the Game!

This code is complete. If your model is trained, you can run this cell to have the model play against itself.

In [45]:
import random
import os

def pick_random_image(class_name):
    folder = os.path.join(DATA_DIR, class_name)
    files = os.listdir(folder)
    img = random.choice(files)
    return os.path.join(folder, img)

def rps_winner(move1, move2):
    if move1 == move2:
        return "Draw"

    rules = {
        "rock": "scissors",
        "paper": "rock",
        "scissors": "paper"
    }

    if rules[move1] == move2:
        return f"Player 1 wins! {move1} beats {move2}"
    else:
        return f"Player 2 wins! {move2} beats {move1}"


# -----------------------------------------------------------
# 1. Choose any two random classes
# -----------------------------------------------------------

choices = ["rock", "paper", "scissors"]
c1 = random.choice(choices)
c2 = random.choice(choices)

img1_path = pick_random_image(c1)
img2_path = pick_random_image(c2)

print("Randomly selected images:")
print("Image 1:", img1_path)
print("Image 2:", img2_path)


# -----------------------------------------------------------
# 2. Predict their labels using the model
# -----------------------------------------------------------

p1 = predict_image(model, img1_path)
p2 = predict_image(model, img2_path)

print("\nPlayer 1 shows:", p1)
print("Player 2 shows:", p2)

# -----------------------------------------------------------
# 3. Decide the winner
# -----------------------------------------------------------

print("\nRESULT:", rps_winner(p1, p2))

Randomly selected images:
Image 1: dataset_rps/scissors/8pCggsYVt6kUOza4.png
Image 2: dataset_rps/paper/KOlOsNsrUdQonYpp.png



Player 1 shows: scissors
Player 2 shows: paper

RESULT: Player 1 wins! scissors beats paper
